# Python und MongoDB

Dieses Beispiel setzt einen laufenden MongoDB-Server mit aktivierter Authentifizierung und ein User (mit Namen und Passwort "pkmlp") für die Datenbank "meineDatenbank" auf Localhost voraus. Dieses Programm zeigt die Bearbeitung (**CRUD** von Dokumenten) einer MOngoDB Datenbank aus einem Python-Programm. Zur Kontrolle und zur Verfolgung der einzelnen Schritte dieses Beispiel-Programmes ist weiter ein MongoDB-GUI (z.B. MongoDB Compass, Studio 3t, etc.) sehr empfohlen. Kontrolliere in einem der MongoDB-GUIs oder im Mongo-Terminal ob die Datenbank "meineDatenbank" und der User "pkmlp" vorhanden ist. 

## Erstellen der Verbindung zum Datenbank Server 

Um aus Python auf eine Datenbank (hier die NoSQL-Datenbank MongoDB) zugreifen zu können, muss eine entsprechende Bibliothek (API - Application Programming Interface) eingebunden werden. 

In [ ]:
import pymongo

Definition der Datenbank-Parameter als Konstanten.

In [ ]:
DATABASE_HOST      =  "localhost"
DATABASE_PORT      =  "27017"
DATABASE_USER      =  "pkmlp"
DATABASE_PASSWORT  =  "pkmlp"
DATABASE_NAME      =  "meineDatenbank"
COLLECTION_NAME    =  "meineSammlung"

Als nächstes muss eine Verbindung zur Datenbank (mit den oben definierten Credentials) hergestellt werden.  

In [ ]:
try:
    dbVerbindung = pymongo.MongoClient('mongodb://'+DATABASE_USER+':'+DATABASE_PASSWORT+'@'+
                                       DATABASE_HOST+':'+DATABASE_PORT+'/'+DATABASE_NAME)
    print("\nVerbindung zu MongoDB erstellt\n")
except pymongo.errors.ConnectionFailure as VerbindungsFehler:
    print("\nKeine Verbindung zu MongoDB: ", VerbindungsFehler, " - Programmabbruch\n")

## Auswählen der Datenbank und der Sammlung

Verbindung konnte erstellt werden, nun die Datenbank ...

In [ ]:
db = dbVerbindung[DATABASE_NAME]

... und die Collection in der Datenbank wählen

In [ ]:
sammlung = db[COLLECTION_NAME]

## Kontrolliere im MongoDB-GUI oder im mongo Terminal ob die Collection wirklich leer ist.

Die Collection muss nicht zwingend leer sein. Ist sie es nicht, so muss der Anfangs-Datenbestand berücksichtigt werden. Das heisst, die nachfolgend eingefügten (**C**reate), gelesenen (**R**ead), mutierten (**U**pdate) und gelöschten (**D**elete) Dokumente sind immer auch mit den allenfalls bereits vorhandenen Dokumenten "zu sehen". Dieses Beispiel ist besser zu Interpretieren, wenn die Sammlung vor Beginn leer ist.

Definieren von zwei Dokumenten. Beachte: die Dokumente müssen nicht eine einheitliche Struktur (gleiche Attribute) haben. 

In [ ]:
dokument_1 = { "firstname":"peter",
               "lastname":"Kessler",
               "contactdata":{
                   "email":"peter.kessler@students.ffhs.ch",
                   "mobile":"+41 79 670 01 13",
                   "home":"+41 44 371 75 62"
               }
             }

dokument_2 = { "firstname":"Ursula",
               "lastname":"Deriu",
               "contactdata":{
                   "email":"ursula.deriu@ffhs.ch"
               },
               "role":"Teacher CAS Big Data"
             }


## CRUD - Create, Read, Update, Delete von Dokumenten

### Create - Einfügen von Dokumenten  

Einfügen der einzeln definierten Dokumente in die Datenbank/Collection

In [ ]:
insertedKey = sammlung.insert_one(dokument_1)
print("\nPrimekey des eingefügten Dokumentes: ", insertedKey.inserted_id)

insertedKey = sammlung.insert_one(dokument_2)
print("Primekey des eingefügten Dokumentes: ", insertedKey.inserted_id)


#### Kontrolliere in MongoClient, MongoBooster oder im mongo Terminal die Collection.

War die Sammlung zu Beginn leer, so dürfen jetzt nur die 2 obigen Dokumente drin sein. War die Sammlung zu Beginn nicht leer, so sind jetzt zusätzlich zu den bereits vorhandenen Dokumenten die obigen 2 in der Sammlung.

Da MongoDB automatisch einen eindeutigen Primärschlüssel (\_id) anlegt, können die gleichen Dokumente mehrfach erfasst werden. Will ich das verhindern, so muss der Primärschlüssel beim Einfügen eines Dokumentes ebenfalls angegeben werden.

### Read - Lesen von Dokumenten 

Lesen aller Dokumente aus der Datenbank
Hinweis: Die Reihenfolge der einzelnen Attribute in einer Zeile sind nicht definiert.
Darum müssen Attribute gezielt mit dem Key aus dem Cursor gelesen werden.

In [ ]:
print("Lesen aller Dokumente nach Create")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)

In [ ]:
print("\nLesen aller Dokumente aber nur bestimmte Attribute ausgeben")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument.get("firstname"), dokument.get("lastname"), dokument.get("role", "keine Rolle erfasst"))

Lesen von Dokumenten mit Suchkriterium 'Vorname = Ursula'

In [ ]:
dokumentCursor = sammlung.find({"firstname":"Ursula"})
for dokument in dokumentCursor:
    print(dokument)

Lesen von Dokumenten mit Suchkriterium 'Vorname ungleich Ursula'

In [ ]:
dokumentCursor = sammlung.find({"firstname":{"$ne":"Ursula"}})
for dokument in dokumentCursor:
    print(dokument)

Lesen von Dokumenten mit Suchkriterien 'Vorname = Ursula und Name = Deriu'

In [ ]:
dokumentCursor = sammlung.find({"firstname":"Ursula","lastname":"Deriu"})
for dokument in dokumentCursor:
    print(dokument)

Lesen von Dokumenten mit Suchkriterium 'Name beginnt mit K'

In [ ]:
dokumentCursor = sammlung.find({"lastname":{"$gt":"K","$lt":"L"}})
for dokument in dokumentCursor:
    print(dokument)

### Update - Mutieren (ändern) von Dokumenten

Updaten eines ganz bestimmten Dokumentes in der Sammlung und zur Kontrolle vor- und nachher alles ausgeben.

In [ ]:
print("\nLesen aller Dokumente vor Update")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)
    
print("\nUpdate des Dokumentes: peter --> Peter")
sammlung.update_one({"firstname":"peter"},{"$set":{"firstname":"Peter"}})

print("\nLesen aller Dokumente nach Update")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)

Kontrolliere im MongoDB-GUI oder im mongo Terminal die Collection.

### Delete - Löschen von Dokumenten

Löschen eines Dokumentes aus der Sammlung

In [ ]:
print("\nLesen aller Dokumente vor Delete")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)
    
print("\nLöschen des Dokumentes: Peter Kessler")
anzDelete = sammlung.delete_one({"firstname":"Peter", "lastname":"Kessler"})
print("Anzahl gelöschter Dokumente: ", anzDelete.deleted_count)

print("\nLesen aller Dokumente nach Delete")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)

Löschen aller Dokumente aus der Sammlung

In [ ]:
print("\nLesen aller Dokumente vor Delete all")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)
    
print("\nLöschen aller Dokumente")
anzDelete = sammlung.delete_many({})
print("Anzahl gelöschter Dokumente: ", anzDelete.deleted_count)

print("\nLesen aller Dokumente nach Delete all")
dokumentCursor = sammlung.find()
for dokument in dokumentCursor:
    print(dokument)

Kontrolliere im MongoDB-GUI oder im mongo Terminal ob die Collection wirklich leer ist.

## Löschen der Sammlung sowie Verbindung zur Datenbank beenden

In [ ]:
sammlung.drop()

In [ ]:
dbVerbindung.close()

That's all folks